q-1-2.py

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree, preprocessing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
eps = np.finfo(float).eps

Reading data from csv

In [2]:
df = pd.read_csv('train.csv')

spliting features and class label and dropping numerical data.

In [3]:
Y1 = df.left
X1 = df.drop(['left'],axis='columns')
# axis = 0 means row and axis= 1 means columns. 
# default is 0. 
Z1 = pd.concat([X1,pd.get_dummies(X1['sales'], prefix='sales')],axis='columns')
Z1 = pd.concat([Z1,pd.get_dummies(Z1['salary'], prefix='salary')],axis='columns')
Z1 = Z1.drop(['sales','salary'],axis='columns')
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.2)
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(Z1, Y1,test_size=0.2)
df11 = pd.concat([X_train1,Y_train1],axis=1)

In [4]:
def get_entropy( df ):
    Class = df.keys()[-1]
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = float(df[Class].value_counts()[value])/len(df[Class])
        entropy += -fraction*np.log2(fraction+eps)
    return entropy

#TESTING OF ENTROPY FUNC
# c = pd.DataFrame({'rushit' : [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,11,1,1,1,1,1,1,1,1,1,1]})
# x_ent = get_entropy(c.rushit)
# x_ent

In [5]:
def get_entropy_attr( df,  attribute ):
    Class = df.keys()[-1]
    target_variables = df[Class].unique()  
    variables = df[attribute].unique()
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*np.log2(fraction+eps)
        fraction2 = float(den)/len(df)
        entropy2 += -fraction2*entropy
    return abs(entropy2)

In [6]:
def attr_to_select(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
        IG.append(get_entropy(df)-get_entropy_attr(df,key))
    return df.keys()[:-1][np.argmax(IG)]

In [7]:
def get_subtable(df, node, value):
    return df[df[node] == value].reset_index(drop=True)

In [8]:
class_ent = get_entropy(df)
def find_IG( df, val, attr ):
    left = df[df[attr] < val ].reset_index(drop=True)
    right = df[df[attr] >= val ].reset_index(drop=True)
    left_ent = get_entropy(left)
    right_ent = get_entropy(right)
    return class_ent - ((float(len(left))/(len(df)+eps) * left_ent)+( float(len(right))/(len(df)+eps) * right_ent))

In [9]:
def split_numerical( attr , Y , attr_name):
    max_ig = 0
    max_split = None
    pair = pd.concat([attr, Y], axis='columns')
    pair = pair.sort_values(by =attr_name).reset_index()
    for i in xrange( len(attr)-1):
#         print i, pair[attr_name][i] , pair[attr_name][i+1]
        if pair['left'][i] != pair['left'][i+1]:
            cur_ig = find_IG( pair, float(pair[attr_name][i] + pair[attr_name][i+1])/2 , attr_name )
            if cur_ig > max_ig:
                max_ig = cur_ig
                max_split =  float(pair[attr_name][i] + pair[attr_name][i+1])/2
    return max_split

In [10]:
def change_actual( df, val, attr ):
    df.loc[df[attr] < val, attr ] = 0
    df.loc[df[attr] >= val, attr ] = 1
    return

In [11]:
num_attr = [ 'number_project','last_evaluation', 'satisfaction_level','average_montly_hours','time_spend_company']
split_dict = {}
for attr in num_attr:
    split_val = split_numerical(df[attr], df['left'],attr)
    split_dict[attr] = split_val
print split_dict

{'satisfaction_level': 0.46499999999999997, 'last_evaluation': 0.58, 'average_montly_hours': 287.5, 'time_spend_company': 3.0, 'number_project': 2.5}


In [12]:
def preprocess(df,split_dict):
    for key,value in split_dict.iteritems():
        change_actual(df, value, key)

In [13]:
preprocess(df,split_dict)

In [14]:
Y = df.left
X = df.drop(['left'],axis='columns')
# axis = 0 means row and axis= 1 means columns. 
# default is 0. 

doing One hot Encoding to convert multi-valued features to binary.

In [15]:
Z = pd.concat([X,pd.get_dummies(X['sales'], prefix='sales')],axis='columns')
Z = pd.concat([Z,pd.get_dummies(Z['salary'], prefix='salary')],axis='columns')
Z = Z.drop(['sales','salary'],axis='columns')

In [16]:
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.2)
X_train, X_test, Y_train, Y_test = train_test_split(Z, Y,test_size=0.2)
df1 = pd.concat([X_train,Y_train],axis=1)

In [17]:
class Node:
    def __init__(self, feature):
        self.feature = feature
        self.positive = 0
        self.negative = 0
        self.left = None
        self.right = None

In [18]:
def build_Tree(df):
    if len(df.columns) == 1:
        return None
    node_to_split = attr_to_select(df)
    
    root = Node(node_to_split)
    root.positive = len( df[df['left']==1]['left'] )
    root.negative = len( df[df['left']==0]['left'] )
    
    subtable0 = get_subtable(df,node_to_split,0)
    subtable1 = get_subtable(df,node_to_split,1)
    
    subtable0 = subtable0.drop(node_to_split,axis=1)
    subtable1 = subtable1.drop(node_to_split,axis=1)
    
#     print node_to_split
#     print subtable0
#     print subtable1
#     print subtable0.shape
#     print subtable1.shape
    clValue0,counts0 = np.unique(subtable0['left'],return_counts=True)
    clValue1,counts1 = np.unique(subtable1['left'],return_counts=True)
    
#     print clValue0, counts0
#     print clValue1, counts1
    
    if len(counts0)<=1:
#         print "left"
        pass
#         if len(counts0)==1:
#             if clValue0[0]==0:
#                 root.negative=counts0[0]
#             else:
#                 root.positive=counts0[0]
    else:
#         print "else left"
        root.left = build_Tree(subtable0)
        
    if len(counts1)<=1:
#         print "right"
        pass
#         if len(counts1)==1:
#             if clValue1[0]==0:
#                 root.negative=counts1[0]
#             else:
#                 root.positive=counts1[0]
    else : 
#         print "else right"
        root.right = build_Tree(subtable1)
    
    return root

In [20]:
def rec_predict(df,root,Y1):
#     print len(Y1)
    if root == None:
        return None
    
    if root.right==None and root.left==None:
        Y1.append(1 if root.positive > root.negative else 0)
        return
    
    if root.right==None and df[root.feature] == 1:
        Y1.append(1 if root.positive > root.negative else 0)
        return 
    if root.left == None and df[root.feature] == 0:
        Y1.append(1 if root.positive > root.negative else 0)
        return
    
    if df[root.feature]==0:
        rec_predict(df,root.left,Y1)
    else:
        rec_predict(df,root.right,Y1)
        
def predict(df,root,Y1):
    for col,row in df.iterrows():
        rec_predict(row,root,Y1)

In [21]:
root = build_Tree(df1)

In [22]:

Yp1=[]
# df2 = X_train.iloc[[0]]
predict(X_test,root,Yp1)
# Y1
print confusion_matrix(Y_test,Yp1)
print classification_report(Y_test,Yp1)
print accuracy_score(Y_test, Yp1)

[[1624   81]
 [ 270  273]]
              precision    recall  f1-score   support

           0       0.86      0.95      0.90      1705
           1       0.77      0.50      0.61       543

   micro avg       0.84      0.84      0.84      2248
   macro avg       0.81      0.73      0.76      2248
weighted avg       0.84      0.84      0.83      2248

0.8438612099644128


In [23]:
model = tree.DecisionTreeClassifier()
model.fit(X_train1, Y_train1)
Y_predict = model.predict(X_test1)
print confusion_matrix(Y_test1,Y_predict)
print classification_report(Y_test1,Y_predict)
print accuracy_score(Y_test1, Y_predict)

[[1657   41]
 [  25  525]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      1698
           1       0.93      0.95      0.94       550

   micro avg       0.97      0.97      0.97      2248
   macro avg       0.96      0.97      0.96      2248
weighted avg       0.97      0.97      0.97      2248

0.9706405693950177
